# Structured Output With Tool Use

In [3]:
# Client Setup
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
region = os.getenv("AWS_REGION")

client = boto3.client("bedrock-runtime", region_name=region)
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [4]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    # print(params)
    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [5]:
# Tool Schemas

article_details_schema = {
    "toolSpec": {
        "name": "article_details",
        "description": "This tool should be called with details about an article. It accepts information about the article's title, author, and related topics.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "The title of the article. Can be left empty.",
                    },
                    "author": {
                        "type": "string",
                        "description": "The name of the article's author. Can be left empty.",
                    },
                    "topics": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A list of topics or categories that the article covers. Can be an empty list.",
                    },
                },
            }
        },
    }
}

# flexible way of getting any type of structure out of Claude
to_json_schema = {
    "toolSpec": {
        "name": "to_json",
        "description": "This tool processes any JSON data and can be used for generating structured content, transforming information, or creating any JSON-based output needed for your task.",
        "inputSchema": {
            "json": {"type": "object", "additionalProperties": True}
        },
    }
}

In [6]:
messages = []

add_user_message(
    messages,
    "Write a one-paragraph scholarly article about computer science. Include a title and author name",
)

result = chat(messages)

add_assistant_message(messages, result["text"])

result["text"]

'# The Evolution of Computational Paradigms in Modern Computing\n\nBy Dr. Eleanor R. Harrington\n\nThe field of computer science has undergone remarkable transformation since its theoretical inception in the early 20th century, evolving from mathematical abstractions to tangible systems that permeate virtually every aspect of contemporary society. This metamorphosis has been characterized by concurrent advancements in both theoretical frameworks and practical implementations, with particular acceleration occurring in the domains of artificial intelligence, distributed systems, and quantum computation. These developments have not only expanded the boundaries of computational capability but have fundamentally altered the relationship between humans and machines, introducing novel ethical considerations regarding privacy, autonomy, and the nature of intelligence itself. As we progress further into an era where computational systems increasingly mediate human experience, the discipline fac

## Using a Fixed Schema: 'article_details_schema'

In [12]:
messages = []

add_user_message(messages, 
    f"""
Analyze the article below and extract key data. Then call the article_details tool.

<article_text>
{result["text"]}
</article_text>
""",
)

# Pass the actual tool schema object, not a string
json_result = chat(messages, tools=[article_details_schema], tool_choice="article_details")
json_result

{'parts': [{'toolUse': {'toolUseId': 'tooluse_olZnRyR4QpCiuky3ZvvIGQ',
    'name': 'article_details',
    'input': {'author': 'Dr. Eleanor J. Harrington',
     'title': 'Paradigm Shifts in Modern Computational Theory: A Brief Overview',
     'topics': ['computer science',
      'computational theory',
      'quantum computing',
      'artificial intelligence',
      'multidimensional optimization',
      'interdisciplinary collaboration',
      'large language models',
      'human-centered design',
      'full-stack perspectives']}}}],
 'stop_reason': 'tool_use',
 'text': ''}

## Using a Flexible Schema: 'to_json_schema'

In [7]:
messages = []

add_user_message(messages, 
f"""
Analyze the article below and extract key data. Then call the to_json tool.

<article_text>
{ result['text'] }
</article_text>

When you call to_json, pass in the following structure:
{{
        "title": str # title of the article,
        "author": str # author of the article,
        "topics": List[str] # List of topics mentioned in the article,
        "num_topics": int # Number of topics mentioned
}}
""",)

# we use the double curly brackets to escape the curly brackets which are required to show the JSON schema

# force Claude to use the to_json tool with tool_choice
flexible_result = chat(messages, tools=[to_json_schema], tool_choice="to_json")
flexible_result

{'parts': [{'toolUse': {'toolUseId': 'tooluse_ccaJfgTUR-a-Hv0RUjVkSA',
    'name': 'to_json',
    'input': {'title': 'The Evolution of Computational Paradigms in Modern Computing',
     'author': 'Dr. Eleanor R. Harrington',
     'topics': ['artificial intelligence',
      'distributed systems',
      'quantum computation',
      'ethics',
      'privacy',
      'autonomy',
      'intelligence',
      'human-machine relationship'],
     'num_topics': 8}}}],
 'stop_reason': 'tool_use',
 'text': ''}

We can easily change the structure of the required format as we need directly in the prompt. We don't need to perform any modifications in the `to_json_schema`.